## OOM(Out Of Memory Error)가 해결하기 어려운 이유
- 발생 이유를 찾기 어려움
- 발생 근원지 찾기 어려움
- Error backtraking이 이상한데로...
- 메모리의 이전상황 파악이 어려움

- 보통의 해결 방법: 배치사이즈 줄이기 → GPU clean → Run!

### 해결방법1. GPUUtill 사용하기
- nvidia-smi처럼 GPU 상태 보여주는 모듈
- iter마다 메모리가 늘어나는지 확인
- colab에서 주로 사용

```
!pip insatll GPUtil
import GPUtil
GPUtil.showUtilization()
```



### 해결방법2. torch.cuda.empty_cache()
- backward pass 데이터 쌓인 것 없앨 때 사용
- 사용되지 않은 GPU상 cache 정리
- 가용 메모리 확보
- del(memory free)과는 다름
- reset 대신 쓰기 좋은 함수


```
import torch
from GPUtil import showUtilization as gpu_usage
print("Initial GPU Usage")
gpu_usage()
tensorList = []
# 랜덤값을 cuda gpu에 쌓이게 함
for x in range(10):
  tensorList.append(torch.randn(10000000,10).cuda())
print("GPU Usage after emptying the cache")
torch.cuda.empty_cache()
gpu_usage()
```



### 해결방법3. trainning loop에 tensor로 축적되는 변수 확인
- tensor로 처리된 변수는 GPU 상의 메모리 사용
- loop안에 연산에 있을 때 GPU에 computational graph 생성

- 1-d tensor의 경우 파이썬 기본 객체로 변환하여 처리 (.item, float)

  ```
  total_loss=0
  for x in range(10):
    # assume loss is computed
    iter_loss = torch.randn(3,4).mean()
    iter_loss.requires_grad = True
    total_loss += iter_loss.item()
  ```



### 해결방법4. del 명령어 적절히 사용하기
- 필요가 없어진 변수는 적절한 삭제가 필요
- 파이썬은 메모리 배치 특성상 loop이 끝나도 메모리를 차지함
- loop 나가기 전에 del 해주기


### 해결방법5. 가능한 batch 사이즈 실험해보기
- batch 사이즈를 1로 해서 실험해보기
- 코드가 정갈해짐


```
oom = False
try:
    run_model(batch_size)
except RuntimeError: # Out of memory
    oom = True
if oom:
    for _ in range(batch_size):
        run_model(1)
```



### 해결방법6. torch.no_grad() 사용
- inference 시점에서는 torch.no_grad() 사용
- <backward pass로 인해 쌓이는 메모리>가 안생김

## 이외...
-  CUDNN_STATUS_NOT_INIT: GPU 문제..
- device-side-asser: OOM과 비슷
- 적절한 코드 처리가 필요!

## 기본적으로 주의해야할 것
- colab에서 너무 큰 사이즈는 실행하지 말 것
  - ex. linear, CNN, LSTM
- CNN의 대부분의 에러는 크기가 안 맞아서 생김
  - torchsummary로 사이즈 맞추기
- tensor의 float precision을 16bit로 줄일수 있음